In [1]:
# Importação das bibliotecas 
import numpy as np # Essa biblioteca consiste em objetos chamados de arrays (matrizes), que são multidimensionais
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
import os
import glob # É usado para retornar todos os caminhos de arquivo que correspondem a um padrão específico.
import plotly.graph_objects as go
import cv2
import warnings
from PIL import Image
from PIL import ImageFile
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
#from tensorflow.keras.optimizers import Adam
from keras.optimizers import adam_v2
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.layers import Dense, Flatten , Dropout
from keras.callbacks import EarlyStopping, ReduceLROnPlateau,ModelCheckpoint
warnings.filterwarnings("ignore")
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [6]:
# Carregando Dataset 

root_dir = '../cancer_colo_utero/dataset'
treino_dir = os.path.join(root_dir,'treino') # método para unir um ou mais componentes de caminho de forma inteligente.

tipo1_dir = os.path.join(treino_dir, 'tipo_1')
tipo2_dir = os.path.join(treino_dir, 'tipo_2')
tipo3_dir = os.path.join(treino_dir, 'tipo_3')

treino_tipo1_files = glob.glob(tipo1_dir+'/*.jpg') # Encontra todos os nomes de caminho que correspondem a um padrão especificado de acordo com as regras usadas pelo shell Unix
treino_tipo2_files = glob.glob(tipo2_dir+'/*.jpg')
treino_tipo3_files = glob.glob(tipo3_dir+'/*.jpg')

added_tipo1_files  =  glob.glob(os.path.join(root_dir, "adicionar_tipo_1", "tipo_1")+'/*.jpg')
added_tipo2_files  =  glob.glob(os.path.join(root_dir, "adicionar_tipo_2", "tipo_2")+'/*.jpg')
added_tipo3_files  =  glob.glob(os.path.join(root_dir, "adicionar_tipo_3", "tipo_3")+'/*.jpg')


tipo1_files = treino_tipo1_files + added_tipo1_files
tipo2_files = treino_tipo2_files + added_tipo2_files
tipo3_files = treino_tipo3_files + added_tipo3_files

print(f'''Foram adicionado {len(tipo1_files)} imagens do Tipo 1  
Foram adicionado {len(tipo2_files)} imagens do Tipo 2
Foram adicionado {len(tipo3_files)} imagens do Tipo 3''' )

Foram adicionado 1441 imagens do Tipo 1  
Foram adicionado 4348 imagens do Tipo 2
Foram adicionado 2426 imagens do Tipo 3


In [7]:
# create dataframe of file and labels
files = {'filepath': tipo1_files + tipo2_files + tipo3_files,
          'label': ['Tipo 1']* len(tipo1_files) + ['Tipo 2']* len(tipo2_files) + ['Tipo 3']* len(tipo3_files)}

files_df = pd.DataFrame(files).sample(frac=1, random_state= 1).reset_index(drop=True)
files_df

,filepath,label
0,../cancer_colo_utero/dataset\treino\tipo_1\141...,Tipo 1
1,../cancer_colo_utero/dataset\adicionar_tipo_3\...,Tipo 3
2,../cancer_colo_utero/dataset\adicionar_tipo_1\...,Tipo 1
3,../cancer_colo_utero/dataset\adicionar_tipo_2\...,Tipo 2
4,../cancer_colo_utero/dataset\adicionar_tipo_3\...,Tipo 3
...,...,...
8210,../cancer_colo_utero/dataset\adicionar_tipo_2\...,Tipo 2
8211,../cancer_colo_utero/dataset\adicionar_tipo_3\...,Tipo 3
8212,../cancer_colo_utero/dataset\adicionar_tipo_1\...,Tipo 1
8213,../cancer_colo_utero/dataset\adicionar_tipo_2\...,Tipo 2


In [14]:
# describe the dataframe
files_df.describe()

,filepath,label
count,8215,8215
unique,8215,3
top,../cancer_colo_utero/dataset\adicionar_tipo_2\...,Tipo 2
freq,1,4348


In [15]:
# check for duplicates
len(files_df[files_df.duplicated(subset=['filepath'])])

0